In [41]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB

In [42]:
# Load your dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_ids = pd.read_csv("test.csv")["ID"]
# Analyze your dataset
# ID, Customer_ID, Month, Name, Age, Profession, Number are columns to be dropped
train_data = train_data.drop(columns=["ID", "Customer_ID", "Month", "Name", "Profession", "Number", "Loan_Type"])

print(train_data.head())

# Loan_Type, Credit_Mix, Payment_of_Min_Amount, Payment_Behaviour , and Credit_Score are categorical columns
# Credit_Score is the target column

C:\Users\krish\AppData\Local\Temp\ipykernel_21488\3789247019.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv("train.csv")


  Age Income_Annual  Base_Salary_PerMonth  Total_Bank_Accounts  \
0  51     101583.48                   NaN                    5   
1  23     101926.95             8635.9125                    4   
2  49     158871.12                   NaN                    0   
3  40      60379.28                   NaN                    5   
4  17      50050.83             4085.9025                    9   

   Total_Credit_Cards  Rate_Of_Interest Total_Current_Loans  \
0                   7                10                  4_   
1                   4                 9                   1   
2                   4                 8                   1   
3                   6                18                   3   
4                  10                20                   5   

   Delay_from_due_date Total_Delayed_Payments Credit_Limit  ...  Credit_Mix  \
0                    8                      8         2.89  ...    Standard   
1                   13                      9        10.26  ...   

### Changes can be done here.Filling data with something other than median

In [43]:
# Fill missing values with median in train data
train_data.fillna(train_data.median(numeric_only=True), inplace=True)
print(train_data.head())

  Age Income_Annual  Base_Salary_PerMonth  Total_Bank_Accounts  \
0  51     101583.48           3086.683333                    5   
1  23     101926.95           8635.912500                    4   
2  49     158871.12           3086.683333                    0   
3  40      60379.28           3086.683333                    5   
4  17      50050.83           4085.902500                    9   

   Total_Credit_Cards  Rate_Of_Interest Total_Current_Loans  \
0                   7                10                  4_   
1                   4                 9                   1   
2                   4                 8                   1   
3                   6                18                   3   
4                  10                20                   5   

   Delay_from_due_date Total_Delayed_Payments Credit_Limit  ...  Credit_Mix  \
0                    8                      8         2.89  ...    Standard   
1                   13                      9        10.26  ...   

In [44]:
# Convert relevant columns to numeric after removing any underscores.This was done as I saw a value in Total_Current_Loans with an underscore.
for col in ["Total_Current_Loans", "Current_Debt_Outstanding", "Income_Annual", "Credit_Limit", "Age", "Total_Credit_Cards", "Total_Bank_Accounts", "Delay_from_due_date"]:
    train_data[col] = pd.to_numeric(
        train_data[col].astype(str).str.replace("_", "", regex=False), errors="coerce"
    )
    test_data[col] = pd.to_numeric(
        test_data[col].astype(str).str.replace("_", "", regex=False), errors="coerce"
    )
train_data
# Income_Annual, Base_Salary_PerMonth,Current_Debt_Outstanding,Ratio_Credit_Utilization, Per_Month_EMI, Monthly_Investment

# 'Total_Delayed_Payments', 'Credit_Mix',
#        'Credit_History_Age', 'Payment_of_Min_Amount', 'Monthly_Investment',
#        'Payment_Behaviour', 'Monthly_Balance'],
#       dtype='object')

,Age,Income_Annual,Base_Salary_PerMonth,Total_Bank_Accounts,Total_Credit_Cards,Rate_Of_Interest,Total_Current_Loans,Delay_from_due_date,Total_Delayed_Payments,Credit_Limit,...,Credit_Mix,Current_Debt_Outstanding,Ratio_Credit_Utilization,Credit_History_Age,Payment_of_Min_Amount,Per_Month_EMI,Monthly_Investment,Payment_Behaviour,Monthly_Balance,Credit_Score
0,51,101583.480,3086.683333,5,7,10,4,8,8,2.89,...,Standard,50.93,34.462154,24 Years and 1 Months,No,190.811017,630.0157894388726,Low_spent_Large_value_payments,314.0021934422197,Standard
1,23,101926.950,8635.912500,4,4,9,1,13,9,10.26,...,_,1058.00,39.693812,20 Years and 5 Months,No,70.587681,662.8039273360225,Low_spent_Medium_value_payments,410.1996419555151,Standard
2,49,158871.120,3086.683333,0,4,8,1,8,2,1.17,...,Good,576.48,39.367225,19 Years and 0 Months,No,86.905860,746.8059854204569,Low_spent_Medium_value_payments,742.5141542054829,Standard
3,40,60379.280,3086.683333,5,6,18,3,15,12,6.83,...,Standard,725.39,29.061701,17 Years and 1 Months,NM,90.906385,166.41865803064803,High_spent_Medium_value_payments,473.13562343490486,Standard
4,17,50050.830,4085.902500,9,10,20,5,28,NaN,15.45,...,Bad,3419.10,30.386321,4 Years and 6 Months,Yes,190.445060,56.789441169542684,High_spent_Large_value_payments,401.3557486786916,Poor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,55,114597.040,3086.683333,7,6,4,4,32,8,10.54,...,Good,926.18,26.436313,31 Years and 9 Months,No,225.923762,327.61966834569836,High_spent_Medium_value_payments,633.131903,Poor
79996,28,8227.855,656.654583,6,8,31,-100,61,20,17.60,...,_,2695.38,24.127401,11 Years and 8 Months,NM,7352.000000,49.54415830254037,Low_spent_Medium_value_payments,268.108435,Poor
79997,46,35032.660,2853.388333,10,6,33,7,48,18,2.52,...,Bad,1789.00,25.086176,11 Years and 5 Months,Yes,150.500097,106.73567925309915,Low_spent_Small_value_payments,318.103057,Poor
79998,42,129680.280,10643.690000,8,3,5,2,27,18,16.65,...,_,240.27,33.944094,20 Years and 5 Months,NM,114.165609,567.1798727916067,High_spent_Small_value_payments,643.023518,Standard


In [45]:
#convert credit_history_age to numeric
# Format : 1 Years and 2 Months
train_data["Credit_History_Age"] = train_data["Credit_History_Age"].str.extract("(\d+)").astype(float)
test_data["Credit_History_Age"] = test_data["Credit_History_Age"].str.extract("(\d+)").astype(float)
train_data["Credit_History_Age"]


0        24.0
1        20.0
2        19.0
3        17.0
4         4.0
         ... 
79995    31.0
79996    11.0
79997    11.0
79998    20.0
79999    17.0
Name: Credit_History_Age, Length: 80000, dtype: float64

In [46]:
# Replace infinity values in train data
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
train_data.fillna(train_data.median(numeric_only=True), inplace=True)

# Label encode the target variable in train data
label_encoder = LabelEncoder()
train_data["Credit_Score"] = label_encoder.fit_transform(train_data["Credit_Score"])

In [47]:
# Prepare training features and labels
X_train = train_data.drop(columns="Credit_Score")
y_train = train_data["Credit_Score"]

In [48]:
# Feature engineering in train data
train_data["Debt_Income_Ratio"] = (
    train_data["Current_Debt_Outstanding"] / train_data["Income_Annual"]
)
train_data["Income_Credit_Limit_Ratio"] = (
    train_data["Income_Annual"] / train_data["Credit_Limit"]
)
train_data["Debt_Credit_Limit_Ratio"] = (
    train_data["Current_Debt_Outstanding"] / train_data["Credit_Limit"]
)

In [49]:
# print Monthly_Balance column
# print(X_train["Monthly_Investment"])
# print(X_train.select_dtypes(include=["object"]).columns)
# Replace all str values in Monthly_Balance, Monthly_Investment

X_train["Monthly_Balance"] = pd.to_numeric(
    X_train["Monthly_Balance"].astype(str).str.replace("_", "", regex=False), errors="coerce"
)

X_train["Monthly_Investment"] = pd.to_numeric(
    X_train["Monthly_Investment"].astype(str).str.replace("_", "", regex=False), errors="coerce"
)

# Fill missing values with median for Monthly_Balance and Monthly_Investment
X_train.fillna(X_train.median(numeric_only=True), inplace=True)

# Same for test data
test_data["Monthly_Balance"] = pd.to_numeric(
    test_data["Monthly_Balance"].astype(str).str.replace("_", "", regex=False), errors="coerce"
)

test_data["Monthly_Investment"] = pd.to_numeric(
    test_data["Monthly_Investment"].astype(str).str.replace("_", "", regex=False), errors="coerce"
)

test_data.fillna(test_data.median(numeric_only=True), inplace=True)

# print Monthly_Balance column
print(X_train["Monthly_Investment"], X_train["Monthly_Balance"])

0        630.015789
1        662.803927
2        746.805985
3        166.418658
4         56.789441
            ...    
79995    327.619668
79996     49.544158
79997    106.735679
79998    567.179873
79999    219.890035
Name: Monthly_Investment, Length: 80000, dtype: float64 0        314.002193
1        410.199642
2        742.514154
3        473.135623
4        401.355749
            ...    
79995    633.131903
79996    268.108435
79997    318.103057
79998    643.023518
79999    858.281177
Name: Monthly_Balance, Length: 80000, dtype: float64


In [50]:
# Step 1: Identify numeric and categorical features


X_train.replace("-", np.nan, inplace=True)
X_train.replace("NM", np.nan, inplace=True)

categorical_features = [
    "Credit_Mix",
    "Payment_of_Min_Amount",
    "Payment_Behaviour",
    "Total_Delayed_Payments"
]
X_train[categorical_features] = X_train[categorical_features].astype(str)

numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X_train.select_dtypes(include=["object"]).columns

# Step 2: Handle Missing Values
# Use SimpleImputer to fill in missing values. Strategies: 'mean' for numerical, 'most_frequent' for categorical.

numerical_pipeline = Pipeline(
    [("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

# Define the preprocessing steps for numeric features
# numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="mean"))])

# Define the preprocessing steps for categorical features
# Using OneHotEncoder to convert categorical variables into binary (dummy) variables.
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "encoder",
            OneHotEncoder(handle_unknown="ignore"),
        ),  #! Can change this step for different encoding methods
    ]
)

# Step 3: Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [51]:
#Print the columns
print(X_train.columns)
# Print Numerical columns
print(X_train.select_dtypes(include=["int64", "float64"]).columns)
# Print Categorical columns
print(X_train.select_dtypes(include=["object"]).columns)

Index(['Age', 'Income_Annual', 'Base_Salary_PerMonth', 'Total_Bank_Accounts',
       'Total_Credit_Cards', 'Rate_Of_Interest', 'Total_Current_Loans',
       'Delay_from_due_date', 'Total_Delayed_Payments', 'Credit_Limit',
       'Total_Credit_Enquiries', 'Credit_Mix', 'Current_Debt_Outstanding',
       'Ratio_Credit_Utilization', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Per_Month_EMI', 'Monthly_Investment',
       'Payment_Behaviour', 'Monthly_Balance'],
      dtype='object')
Index(['Age', 'Income_Annual', 'Base_Salary_PerMonth', 'Total_Bank_Accounts',
       'Total_Credit_Cards', 'Rate_Of_Interest', 'Total_Current_Loans',
       'Delay_from_due_date', 'Credit_Limit', 'Total_Credit_Enquiries',
       'Current_Debt_Outstanding', 'Ratio_Credit_Utilization',
       'Credit_History_Age', 'Per_Month_EMI', 'Monthly_Investment',
       'Monthly_Balance'],
      dtype='object')
Index(['Total_Delayed_Payments', 'Credit_Mix', 'Payment_of_Min_Amount',
       'Payment_Behaviour'],
  

In [52]:
# Feature engineering in test data
test_data["Debt_Income_Ratio"] = (
    test_data["Current_Debt_Outstanding"] / test_data["Income_Annual"]
)
test_data["Income_Credit_Limit_Ratio"] = (
    test_data["Income_Annual"] / test_data["Credit_Limit"]
)
test_data["Debt_Credit_Limit_Ratio"] = (
    test_data["Current_Debt_Outstanding"] / test_data["Credit_Limit"]
)

In [53]:
# Check for mixed data types in each column
for col in X_train.columns:
    unique_types = set(type(x) for x in X_train[col].dropna())
    if len(unique_types) > 1:
        print(f"Column '{col}' has mixed types: {unique_types}")

# # Convert all values to NaN whenever str is encountered for Monthly_Balance
# X_train["Monthly_Balance"] = pd.to_numeric(X_train["Monthly_Balance"], errors="coerce")

# # Fill missing values in 'Monthly_Balance' with the median value
# X_train["Monthly_Balance"].fillna(X_train["Monthly_Balance"].median(), inplace=True)


# X_train["Monthly_Balance"] = pd.to_numeric(X_train["Monthly_Balance"], errors="coerce")
# # test_data["Monthly_Balance"] = pd.to_numeric(
# #     test_data["Monthly_Balance"], errors="coerce"
# # )

# # Fill missing values in 'Monthly_Balance' with the median value
# X_train["Monthly_Balance"].fillna(X_train["Monthly_Balance"].median(), inplace=True)
# # test_data["Monthly_Balance"].fillna(test_data["Monthly_Balance"].median(), inplace=True)

In [54]:
print(X_train.select_dtypes(include=["int64", "float64"]).columns)

Index(['Age', 'Income_Annual', 'Base_Salary_PerMonth', 'Total_Bank_Accounts',
       'Total_Credit_Cards', 'Rate_Of_Interest', 'Total_Current_Loans',
       'Delay_from_due_date', 'Credit_Limit', 'Total_Credit_Enquiries',
       'Current_Debt_Outstanding', 'Ratio_Credit_Utilization',
       'Credit_History_Age', 'Per_Month_EMI', 'Monthly_Investment',
       'Monthly_Balance'],
      dtype='object')


In [55]:
# Step 4: Apply preprocessing

# Apply transformations to the training data
X_preprocessed = preprocessor.fit_transform(X_train)

# Convert to DataFrame and assign column names
X_train_df = pd.DataFrame.sparse.from_spmatrix(X_preprocessed)
X_train_df.columns = preprocessor.get_feature_names_out()

# Apply transformations to the test data and convert to DataFrame with column names
test_data_preprocessed = preprocessor.transform(test_data)
test_data_df = pd.DataFrame.sparse.from_spmatrix(test_data_preprocessed)
test_data_df.columns = preprocessor.get_feature_names_out()

# Step 5: Train-Test Split for the preprocessed data
X_train_1, X_val_1, y_train_1, y_val = train_test_split(
    X_train_df, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Verify the transformations
print("Transformed Training Data Shape:", X_train_1.shape)
print("Transformed Validation Data Shape:", X_val_1.shape)
print("Transformed Test Data Shape:", test_data_df.shape)

Transformed Training Data Shape: (64000, 646)
Transformed Validation Data Shape: (16000, 646)
Transformed Test Data Shape: (20000, 646)


In [56]:
models = {}
# model = RandomForestClassifier(n_estimators=300, max_depth=6, random_state=42)
# models["Random forest"] = model
# model = XGBClassifier(
#     learning_rate=0.05,
#     max_depth=6,
#     n_estimators=300,
#     random_state=42,
#     eval_metric="mlogloss",
# )
model = XGBClassifier(
    learning_rate=0.17031088174537234,
    max_depth=9,
    n_estimators=177,
    random_state=42,
    eval_metric="mlogloss",
)
models["XGBoost"] = model
# model = KNeighborsClassifier(n_neighbors=5)
# models["K nearest neighbours"] = model
# model = LogisticRegression(random_state=42, max_iter=500)
# models["Logistic regression"] = model
# model = DecisionTreeClassifier(max_depth=6, random_state=42)
# models["Decision tree classifier"] = model
# model = GaussianNB()
# models["Gaussian"] = model
# base_estimator = DecisionTreeClassifier(max_depth=6)
# model = AdaBoostClassifier(estimator=base_estimator, n_estimators=300, random_state=42)
# models["Adaboost"] = model

In [57]:
y_train

0        2
1        2
2        2
3        2
4        1
        ..
79995    1
79996    1
79997    1
79998    2
79999    2
Name: Credit_Score, Length: 80000, dtype: int64

In [58]:
print(X_train_1.columns)
print(y_train)
# y_train should have 1 column of credit score


Index(['num__Age', 'num__Income_Annual', 'num__Base_Salary_PerMonth',
       'num__Total_Bank_Accounts', 'num__Total_Credit_Cards',
       'num__Rate_Of_Interest', 'num__Total_Current_Loans',
       'num__Delay_from_due_date', 'num__Credit_Limit',
       'num__Total_Credit_Enquiries',
       ...
       'cat__Payment_of_Min_Amount_No', 'cat__Payment_of_Min_Amount_Yes',
       'cat__Payment_of_Min_Amount_nan', 'cat__Payment_Behaviour_!@9#%8',
       'cat__Payment_Behaviour_High_spent_Large_value_payments',
       'cat__Payment_Behaviour_High_spent_Medium_value_payments',
       'cat__Payment_Behaviour_High_spent_Small_value_payments',
       'cat__Payment_Behaviour_Low_spent_Large_value_payments',
       'cat__Payment_Behaviour_Low_spent_Medium_value_payments',
       'cat__Payment_Behaviour_Low_spent_Small_value_payments'],
      dtype='object', length=646)
0        2
1        2
2        2
3        2
4        1
        ..
79995    1
79996    1
79997    1
79998    2
79999    2
Name: Cred

In [40]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score


# Objective function for Optuna
def objective(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
    }

    # Initialize the classifier with trial parameters
    model = XGBClassifier(**params, random_state=42, eval_metric="mlogloss")

    # 5-fold cross-validation
    accuracy = cross_val_score(model, X_train_1, y_train_1, cv=5, scoring="accuracy").mean()
    return accuracy


# Run Bayesian Optimization with Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# [I 2024-11-03 16:53:08,619] Trial 0 finished with value: 0.7062031249999999 and parameters: {'learning_rate': 0.050893568127103876, 'max_depth': 4, 'n_estimators': 268, 'subsample': 0.6738283278380766, 'colsample_bytree': 0.6915738416826347}. Best is trial 0 with value: 0.7062031249999999.

# [I 2024-11-03 16:58:00,159] Trial 2 finished with value: 0.7632656250000001 and parameters: {'learning_rate': 0.17031088174537234, 'max_depth': 9, 'n_estimators': 177, 'subsample': 0.8174475788946926, 'colsample_bytree': 0.5291010066051618}. Best is trial 2 with value: 0.7632656250000001.

# Best parameters and score
print("Best parameters found: ", study.best_params)
print("Best accuracy: ", study.best_value)

c:\Users\krish\OneDrive-MSFT\Subjects5thSemester\Machine Learning(Course)\ML-course-project-credit-rank\mlenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-11-03 16:50:27,154] A new study created in memory with name: no-name-ef14d1a9-4bcf-4af6-8f8d-ff78e69e363f
c:\Users\krish\OneDrive-MSFT\Subjects5thSemester\Machine Learning(Course)\ML-course-project-credit-rank\mlenv\Lib\site-packages\xgboost\data.py:575: UserWarning: Sparse arrays from pandas are converted into dense.
  warnings.warn("Sparse arrays from pandas are converted into dense.")
c:\Users\krish\OneDrive-MSFT\Subjects5thSemester\Machine Learning(Course)\ML-course-project-credit-rank\mlenv\Lib\site-packages\xgboost\data.py:575: UserWarning: Sparse arrays from pandas are converted into dense.
  warnings.warn("Sparse arrays from pandas are convert

KeyboardInterrupt: 

In [59]:
for key, value in models.items():
    try:  
        # Define model pipeline
        pipeline = Pipeline([("preprocessor", preprocessor), ("classifier", value)])

        # Fit the pipeline on training data
        pipeline.fit(X_train, y_train)
        
        # Make predictions on test data
        test_predictions = pipeline.predict(test_data)

        # Convert predictions back to original labels
        test_predictions_labels = label_encoder.inverse_transform(test_predictions)

        # test_predictions_encoded = label_encoder.transform(test_predictions)
        # Prepare the submission file
        submission = pd.DataFrame(
            {"ID": test_ids, "Credit_Score": test_predictions_labels}
        )
        submission.to_csv(f"submission_{key}.csv", index=False)

        print(f"Submission file 'submission_{key}.csv' created successfully!")
    except Exception as e:
        print(f"Error : {e}")

Submission file 'submission_XGBoost.csv' created successfully!
